In [11]:
%run ../app/models/model.py
from owlready2 import *
onto = get_ontology("../app/data/education.owl").load()

In [12]:
print("sử dụng SPARQL để tìm tất cả các học sinh có họ Lê")
result = list(default_world.sparql("""
PREFIX s: <http://hc.com/school#>
SELECT ?hs 
WHERE {
    ?hs a s:HocSinh.
    ?hs s:hoTen ?hoTen.
    FILTER (STRSTARTS(?hoTen, "Lê")).
}
"""))
for x in result: print(x[0].hoTen)

sử dụng SPARQL để tìm tất cả các học sinh có họ Lê
Lê Thanh Minh
Lê Ngọc Sơn


In [13]:
print("sử dụng SPARQL để tìm tất cả các học sinh sinh vào tháng 3")
result = list(default_world.sparql("""
PREFIX s: <http://hc.com/school#>
SELECT ?hs 
WHERE {
    ?hs a s:HocSinh.
    ?hs s:ngaySinh ?ngaySinh.
    FILTER (REGEX(?ngaySinh, ".*/03/.*", "i")).
}
"""))
for x in result: print(x[0].hoTen, x[0].ngaySinh)

sử dụng SPARQL để tìm tất cả các học sinh sinh vào tháng 3
Ngô Mai Việt 19/03/2000
Nguyễn Hoài Linh 31/03/2000
Hồ Hữu Dũng 16/03/2000


In [14]:
print("sử dụng SPARQL để tìm tất cả các học sinh của lớp có tên là 10C03")
result = list(default_world.sparql("""
PREFIX s: <http://hc.com/school#>
SELECT ?hs 
WHERE {
    ?hs a s:HocSinh.
    ?hs s:hocLop ?lop.
    ?lop s:ten "10C03".
}
"""))
for x in result: print(x[0].hoTen, x[0].hocLop.ten)

sử dụng SPARQL để tìm tất cả các học sinh của lớp có tên là 10C03
Tăng Thu Tùng 10C03
Dương Thị Cường 10C03
Tôn Mai Việt 10C03
Quách Mai Thiện 10C03
Ngô Mai Việt 10C03


In [15]:
print("sử dụng SPARQL để tìm tất cả các học sinh của lớp có uri là #lophoc1")
result = list(default_world.sparql("""
PREFIX s: <http://hc.com/school#>
SELECT ?hs 
WHERE {
    ?hs a s:HocSinh.
    ?hs s:hocLop ?lop.
    FILTER(STRENDS(STR(?lop), "#lophoc1")).
}
"""))
for x in result: print(x[0].hoTen, x[0].hocLop.ten)

sử dụng SPARQL để tìm tất cả các học sinh của lớp có uri là #lophoc1
Bùi Hữu Nam 10C01
Tăng Thanh Hoàng 10C01
Mai Kim Dũng 10C01
Đào Anh Xuân 10C01
Lê Thanh Minh 10C01


In [21]:
print("sử dụng SPARQL để tìm tất cả các giáo viên chủ nhiệm của từng lớp, sắp xếp theo tên lớp")
result = list(default_world.sparql("""
PREFIX s: <http://hc.com/school#>
SELECT ?gv
WHERE {
    ?gv a s:GiaoVien.
    ?gv s:lopChuNhiem ?lop.
    ?lop s:ten ?tenLop.
    FILTER(BOUND(?lop)).
}
ORDER BY ?tenLop
"""))
for x in result: print(' - '.join([x[0].hoTen, x[0].dayMon[0].ten, x[0].lopChuNhiem.ten]))

sử dụng SPARQL để tìm tất cả các giáo viên chủ nhiệm của từng lớp, sắp xếp theo tên lớp
Đoàn Mai Quang - Hóa học - 10C01
Tạ Hồng Tú - Lịch sử - 10C02
Đào Bảo Ngọc - Hóa học - 10C03
Dương Bảo Dũng - Toán học - 11B01
Lý Bảo Quân - Hóa học - 11B02
Quách Hữu Cường - Sinh học - 11B03
Tăng Hoài Hiền - Lịch sử - 12A01
Đinh Thị Cường - Toán học - 12A02
Trương Thu Quang - Tiếng Anh - 12A03


In [29]:
print("sử dụng SPARQL để tìm tất cả các giáo viên chủ nhiệm của từng lớp (cách 2)")
result = list(default_world.sparql("""
PREFIX s: <http://hc.com/school#>
SELECT ?lop 
WHERE {
    ?lop a s:LopHoc.
}
"""))
for x in result: print(' - '.join([x[0].ten, x[0].giaoVienChuNhiem.hoTen]))

sử dụng SPARQL để tìm tất cả các giáo viên chủ nhiệm của từng lớp (cách 2)
10C01 - Đoàn Mai Quang
10C02 - Tạ Hồng Tú
10C03 - Đào Bảo Ngọc
11B01 - Dương Bảo Dũng
11B02 - Lý Bảo Quân
11B03 - Quách Hữu Cường
12A01 - Tăng Hoài Hiền
12A02 - Đinh Thị Cường
12A03 - Trương Thu Quang


In [44]:
print("sử dụng SPARQL để tìm tất cả các phụ huynh có 2 con đang học cùng trường")
result = list(default_world.sparql("""
PREFIX s: <http://hc.com/school#>
SELECT ?ph (COUNT(?hs) AS ?soCon)
WHERE {
    ?ph a s:PhuHuynh.
    ?ph s:con ?hs.
}
GROUP BY ?ph
HAVING (COUNT(?hs) = 2)
"""))
lst = list(map(lambda ph: [ph[0].hoTen, ph[0].con[0].hoTen, ph[0].con[1].hoTen], result))# Lấy họ tên phụ huynh và các con
lst.sort(key = lambda x:x[2]) # Sắp xếp theo tên con thứ 2
new_lst = [[a[0], b[0], a[1], a[2]] for a, b in zip(lst[::2], lst[1::2])] # Ghép lại theo cặp
for x in new_lst: print(f"({x[0]} + {x[1]}) = Con 1: {x[2]} - Con 2: {x[3]}")

sử dụng SPARQL để tìm tất cả các phụ huynh có 2 con đang học cùng trường
(Đào Hồng Quân + Dương Hoài Xuân) = Con 1: Đào Anh Xuân - Con 2: Bành Anh Quân
(Huỳnh Kim Linh + Phan Thị Thuận) = Con 1: Đặng Quý Thiện - Con 2: Bành Thu Quỳnh
(Trương Văn Long + Quách Mai Thảo) = Con 1: Huỳnh Đình Thiên - Con 2: Dương Thị Cường
(Lương Văn Nam + Lê Hồng Minh) = Con 1: Tăng Bảo Khánh - Con 2: Hoàng Quý Thiên
(Hồ Văn Thúy + Võ Kim Phương) = Con 1: Lâm Hồng Nhật - Con 2: Hồ Hữu Dũng
(Đào Kim Ngọc + Đinh Quý Quang) = Con 1: Lê Ngọc Sơn - Con 2: Hồ Mai Tùng
(Nguyễn Bảo Phương + Trương Minh Quân) = Con 1: Quách Hoài Khánh - Con 2: Lâm Đình Hùng
(Tô Thu Thúy + Tạ Kim Vinh) = Con 1: Nguyễn Hoài Linh - Con 2: Lý Văn Long
(Hồ Minh Thiên + Lâm Bảo Dũng) = Con 1: Tăng Thanh Hoàng - Con 2: Mai Kim Dũng
(Hoàng Đình Thiện + Tăng Minh Tú) = Con 1: Quách Mai Thiện - Con 2: Ngô Mai Việt
(Lê Hồng Nhật + Tôn Mai Thúy) = Con 1: Đào Thu Xuân - Con 2: Phan Mai Quân
(Trương Quý Vinh + Đoàn Kim Tú) = Con 1: Tôn Mai Việt 